<center><a href="https://www.nvidia.com/dli"> <img src="https://github.com/sathyasheelans/NVIDIA_DL/blob/master/Nvidia%20-%20Intro%20to%20Deep%20Learning/Lesson_1/images/DLI_Header.png?raw=1" alt="Header" style="width: 400px;"/> </a></center>

# 1. Image Classification with the MNIST Dataset

Create a model in both pytorch and and tensorflow and compare the differences

## 1.1 Objectives

### Install pytorch and tensorflow

In [1]:
!pip3 install torch torchvision torchaudio

### Load Pytorch

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

# Visualization tools
import torchvision
import torchvision.transforms.v2 as transforms
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

In [ ]:
train_set = torchvision.datasets.MNIST("./data/", train=True, download=True)
valid_set = torchvision.datasets.MNIST("./data/", train=False, download=True)

We stated above that the MNIST dataset contained 70,000 grayscale images of handwritten digits. By executing the following cells, we can see that Keras has partitioned 60,000 of these [PIL Images](https://pillow.readthedocs.io/en/stable/reference/Image.html) for training, and 10,000 for validation (after training).

## 1.4 Preparing the Data for Training

In [ ]:
trans = transforms.Compose([transforms.ToTensor()])

In [ ]:
train_set.transform = trans
valid_set.transform = trans

In [ ]:
batch_size = 32

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size)

## 1.5 Creating the Model

In [ ]:
layers = []
layers

In [ ]:
layers = [
    nn.Flatten()
]
layers

### 1.5.2 The Input Layer

In [ ]:
input_size = 1 * 28 * 28

In [ ]:
layers = [
    nn.Flatten(),
    nn.Linear(input_size, 512),  # Input
    nn.ReLU(),  # Activation for input
]
layers

### 1.5.3 The Hidden Layer

In [ ]:
layers = [
    nn.Flatten(),
    nn.Linear(input_size, 512),  # Input
    nn.ReLU(),  # Activation for input
    nn.Linear(512, 512),  # Hidden
    nn.ReLU()  # Activation for hidden
]
layers

In [ ]:
n_classes = 10

layers = [
    nn.Flatten(),
    nn.Linear(input_size, 512),  # Input
    nn.ReLU(),  # Activation for input
    nn.Linear(512, 512),  # Hidden
    nn.ReLU(),  # Activation for hidden
    nn.Linear(512, n_classes)  # Output
]
layers

### 1.5.5 Compiling the Model

In [ ]:
model = nn.Sequential(*layers)
model

In [ ]:
model.to(device)

In [ ]:
next(model.parameters()).device

In [ ]:
model = torch.compile(model)

## 1.6 Training the Model

In [ ]:
loss_function = nn.CrossEntropyLoss()

In [ ]:
optimizer = Adam(model.parameters())

### 1.6.2 Calculating Accuracy

In [ ]:
train_N = len(train_loader.dataset)
valid_N = len(valid_loader.dataset)
train_N

In [ ]:
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

### 1.6.3 The Train Function

In [ ]:
def train():
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    print('Train - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

### 1.6.4 The Validate Function

In [ ]:
def validate():
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    print('Valid - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

### 1.6.5 The Training Loop

In [ ]:
epochs = 5

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train()
    validate()

In [ ]:
prediction = model(x_0_gpu)
prediction

In [ ]:
prediction.argmax(dim=1, keepdim=True)

Did it get it right?

### 1.7.1 Clear the Memory

Before moving on, please execute the following cell to clear up the GPU memory. This is required to move on to the next notebook.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<center><a href="https://www.nvidia.com/dli"> <img src="https://github.com/sathyasheelans/NVIDIA_DL/blob/master/Nvidia%20-%20Intro%20to%20Deep%20Learning/Lesson_1/images/DLI_Header.png?raw=1" alt="Header" style="width: 400px;"/> </a></center>